In [1]:
import pandas as pd

df = pd.read_csv("match_stats.csv")


In [6]:
# Xem số dòng, số cột
print(f"Dữ liệu có {df.shape[0]} dòng và {df.shape[1]} cột")

# Xem vài dòng đầu
df.head()


Dữ liệu có 26882 dòng và 9 cột


,teamId,teamPosition,championName,win,kills,deaths,assists,visionScore,matchId
0,100,TOP,Gnar,False,1,6,3,14,SG2_79062074
1,100,JUNGLE,Pantheon,False,6,8,3,26,SG2_79062074
2,100,MIDDLE,Ryze,False,2,6,10,17,SG2_79062074
3,100,BOTTOM,Jhin,False,5,5,8,11,SG2_79062074
4,100,UTILITY,Nautilus,False,1,6,8,52,SG2_79062074


In [3]:
# Đếm số lượng giá trị null theo từng cột
df.isnull().sum()


teamId             0
teamPosition    5918
championName       0
win                0
kills              0
deaths             0
assists            0
visionScore        0
matchId            0
dtype: int64

In [5]:
# Xóa các dòng bị thiếu 'position'
df = df.dropna(subset=['teamPosition'])

# In số dòng sau khi lọc
print("Sau khi lọc:", df.shape)

# Ghi dữ liệu đã làm sạch ra file mới
df.to_csv('cleaned_data.csv', index=False)

Sau khi lọc: (26882, 9)


In [9]:
import pandas as pd

# Load dữ liệu
df = pd.read_csv("cleaned_data.csv")  # đổi path nếu cần

# --- 1. Chuyển cột 'win' từ True/False sang 1/0 ---
df['win'] = df['win'].astype(int)

# --- 2. Tạo thêm cột KDA và tham gia hạ gục ---
df['KDA'] = (df['kills'] + df['assists']) / df['deaths'].replace(0, 1)
df['kill_participation'] = df['kills'] + df['assists']

# --- 3. Điền thiếu teamPosition bằng vị trí phổ biến nhất của từng championName ---
champion_position_mode = (
    df[df['teamPosition'].notna()]
    .groupby('championName')['teamPosition']
    .agg(lambda x: x.mode().iloc[0])
)

# Điền vào các dòng bị thiếu teamPosition
df['teamPosition'] = df.apply(
    lambda row: champion_position_mode[row['championName']] if pd.isna(row['teamPosition']) else row['teamPosition'],
    axis=1
)

# --- 4. Tỷ lệ thắng theo tướng ---
win_rate_by_champion = df.groupby('championName')['win'].mean().sort_values(ascending=False)

# --- 5. Tỷ lệ thắng theo vị trí ---
win_rate_by_position = df.groupby('teamPosition')['win'].mean().sort_values(ascending=False)

# --- 6. Điểm tầm nhìn trung bình theo win/thua ---
vision_score_by_win = df.groupby('win')['visionScore'].mean()

# --- (Gợi ý) 7. Thêm cột gold_efficiency nếu bạn có goldEarned và damage ---
# if 'goldEarned' in df.columns and 'totalDamageDealtToChampions' in df.columns:
#     df['gold_efficiency'] = df['totalDamageDealtToChampions'] / df['goldEarned']

# Lưu lại để import lên QuickSight nếu muốn
df.to_csv("match_data_cleaned.csv", index=False)

In [10]:
df.head()

,teamId,teamPosition,championName,win,kills,deaths,assists,visionScore,matchId,KDA,kill_participation
0,100,TOP,Gnar,0,1,6,3,14,SG2_79062074,0.666667,4
1,100,JUNGLE,Pantheon,0,6,8,3,26,SG2_79062074,1.125000,9
2,100,MIDDLE,Ryze,0,2,6,10,17,SG2_79062074,2.000000,12
3,100,BOTTOM,Jhin,0,5,5,8,11,SG2_79062074,2.600000,13
4,100,UTILITY,Nautilus,0,1,6,8,52,SG2_79062074,1.500000,9
